In [12]:
import smtplib
import imaplib
import email
import speech_recognition as sr
import pyttsx3
from email.message import EmailMessage
from email.header import decode_header
from PIL import Image
import io
from transformers import BlipProcessor, BlipForConditionalGeneration

# Initialize speech recognition and text-to-speech engines
listener = sr.Recognizer()
engine = pyttsx3.init()

# Predefined email credentials
email_address = 'sriharshithadurgam29@gmail.com'
password = 'sqtv klza wuot neiw'

# Function to speak out text
def talk(text):
    engine.say(text)
    engine.runAndWait()

# Function to get speech input
def get_info(prompt):
    try:
        with sr.Microphone() as source:
            listener.adjust_for_ambient_noise(source, duration=0.5)
            talk(prompt)
            print(prompt)
            voice = listener.listen(source, timeout=5, phrase_time_limit=10)
            info = listener.recognize_google(voice)
            print(info)
            return info.lower()
    except sr.UnknownValueError:
        talk("Sorry, I couldn't understand what you said. Please try again.")
    except sr.RequestError:
        talk("Sorry, I'm unable to access the Google API at the moment. Please try again later.")
    except sr.WaitTimeoutError:
        talk("Sorry, I didn't hear anything. Please try again.")
    return ""

# Function to send email
def send_email(receiver, subject, message):
    try:
        server = smtplib.SMTP('smtp.gmail.com', 587)
        server.starttls()
        server.login(email_address, password)

        email_msg = EmailMessage()
        email_msg['From'] = email_address
        email_msg['To'] = receiver
        email_msg['Subject'] = subject
        email_msg.set_content(message)

        server.send_message(email_msg)
        server.quit()

        talk("Your email has been sent successfully.")
    except smtplib.SMTPAuthenticationError:
        talk("Sorry, authentication failed. Please check your email and password.")
    except smtplib.SMTPException as e:
        talk(f"Sorry, an error occurred while sending the email: {e}")

# Function to describe an image using a pre-trained model
def describe_image(image):
    try:
        processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
        model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")

        inputs = processor(images=image, return_tensors="pt")
        out = model.generate(**inputs)
        description = processor.decode(out[0], skip_special_tokens=True)
        return description
    except Exception as e:
        print(f"Error in describing image: {e}")
        return None

# Function to read emails
def read_emails():
    try:
        mail = imaplib.IMAP4_SSL("imap.gmail.com")
        mail.login(email_address, password)

        mail.select("inbox")
        status, messages = mail.search(None, "ALL")

        email_ids = messages[0].split()
        latest_email_id = email_ids[-1]
        status, msg_data = mail.fetch(latest_email_id, "(RFC822)")

        for response_part in msg_data:
            if isinstance(response_part, tuple):
                msg = email.message_from_bytes(response_part[1])
                subject, encoding = decode_header(msg["Subject"])[0]
                if isinstance(subject, bytes):
                    subject = subject.decode(encoding if encoding else "utf-8")

                from_ = msg.get("From")
                talk(f"Subject: {subject}")
                talk(f"From: {from_}")

                if msg.is_multipart():
                    for part in msg.walk():
                        content_type = part.get_content_type()
                        content_disposition = str(part.get("Content-Disposition"))

                        if "attachment" in content_disposition:
                            filename = part.get_filename()
                            if filename:
                                print(f"Attachment found: {filename}")
                                talk(f"Attachment: {filename}")
                                if any(image_type in content_type for image_type in ["image/jpeg", "image/png"]):
                                    image_data = part.get_payload(decode=True)
                                    try:
                                        image = Image.open(io.BytesIO(image_data))
                                        print(f"Image opened: {image}")
                                        description = describe_image(image)
                                        if description:
                                            talk(f"The image contains: {description}")
                                        else:
                                            talk("The image content could not be identified.")
                                    except Exception as e:
                                        print(f"Error in opening image: {e}")
                        elif content_type == "text/plain":
                            body = part.get_payload(decode=True).decode()
                            talk(f"Body: {body}")
                else:
                    content_type = msg.get_content_type()
                    if content_type == "text/plain":
                        body = msg.get_payload(decode=True).decode()
                        talk(f"Body: {body}")

        mail.close()
        mail.logout()
    except imaplib.IMAP4.error as e:
        talk(f"Sorry, an error occurred while reading emails: {e}")

# Function to interactively send an email
def get_email_info():
    talk('Hi, I am your assistant for today. To whom do you want to send an email?')

    name = get_info("Please say the recipient's name. speak")
    talk(name)
    email_list = {
        'sonu': 'sriharshithadurgam29@gmail.com',
        'harshi': 'harshithadurgam29112004@gmail.com',
        'ammulu': '23AG1A1249@gmail.com',
    }
    if name in email_list:
        receiver = email_list[name]
        subject = get_info("What is the subject of your email? speak")
        talk("your subject is")
        talk(subject)
        message = get_info("Tell me the text in your email speak")
        talk("your text is")
        talk(message)

        send_email(receiver, subject, message)

        send_more = get_info("Do you want to send another email? speak")
        talk("you said")
        talk(send_more)
        if 'yes' in send_more:
            get_email_info()
        else:
            talk("Goodbye! Have a great day.")
    else:
        talk("Sorry, I couldn't find the recipient in the email list.")

# Main function to initiate the assistant
def main():
    talk("hii boss")
    talk("Do you want to send an email or read an email?")
    action = get_info("Please say 'send' to send an email or 'read' to read an email.")
    talk(action)
    print(f"Action: {action}")

    if 'send' in action:
        print("send mail")
        get_email_info()
    elif 'read' in action:
        print("read email")
        read_emails()
    else:
        print("Didn't understand the action")
        talk("Sorry, I didn't understand. Please say 'send' to send an email or 'read' to read an email.")

if __name__=='__main__':
    main()

Please say 'send' to send an email or 'read' to read an email.
send
Action: send
send mail
Please say the recipient's name. speak
Rishabh dot Bafna 2@gmail.com
